In [9]:
from delta import  # Delta Lake 라이브러리 임포트 configure_spark_with_delta_pip, DeltaTable
from pyspark.sql import SparkSession  # Spark SQL 작업을 위한 SparkSession 임포트

In [10]:
builder = (SparkSession.builder  # SparkSession 빌더 패턴 시작
           .appName("create-delta-table")  # 애플리케이션 이름 설정
           .master("spark://spark-master:7077")  # Spark 마스터 URL 설정
           .config("spark.executor.memory", "512m")  # Spark 설정 옵션   
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")  # Spark 설정 옵션
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")  # Spark 설정 옵션)

spark = configure_spark_with_delta_pip(builder).getOrCreate()  # SparkSession 생성 또는 기존 세션 반환
spark.sparkContext.setLogLevel("ERROR")  # 로그 레벨을 ERROR로 설정

In [11]:
get_ipython().run_line_magic('load_ext', 'sparksql_magic')
get_ipython().run_line_magic('config', 'SparkSql.limit=20')

The sparksql_magic extension is already loaded. To reload it, use:
  %reload_ext sparksql_magic


In [12]:
%%sparksql
CREATE OR REPLACE TABLE default.netflix_titles (
    show_id STRING,
    type STRING,
    title STRING,
    director STRING,
    cast STRING,
    country STRING,
    date_added STRING,
    release_year STRING,
    rating STRING,
    duration STRING,
    listed_in STRING,
    description STRING 
) USING DELTA  # Delta Lake 테이블 생성 LOCATION '/opt/workspace/data/delta_lake/netflix_titles';

In [13]:
# 읽기 CSV file into a DataFrame
df = (spark.read
      .format("csv")
      .option("header", "true")  # 첫 번째 행을 헤더로 사용
      .load(  # 파일 로드"../data/netflix_titles.csv"))

In [14]:
df.printSchema()  # DataFrame 스키마 구조 출력

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



In [15]:
df.write.format("delta")  # Delta Lake 형식으로 저장.mode("overwrite").saveAsTable("default.netflix_titles")

In [16]:
%%sparksql 
SELECT * FROM default.netflix_titles LIMIT 3;

In [17]:
spark.stop()  # Spark 세션 종료 - 리소스 정리